# ലളിതമായ ഇമേജ് ക്ലാസിഫയർ

ഈ നോട്ട്‌ബുക്ക് പ്രീ-ട്രെയിൻ ചെയ്ത ന്യൂറൽ നെറ്റ്‌വർക്ക് ഉപയോഗിച്ച് ചിത്രങ്ങൾ എങ്ങനെ ക്ലാസിഫൈ ചെയ്യാമെന്ന് കാണിക്കുന്നു.

**നിങ്ങൾ പഠിക്കാനിരിക്കുന്നതെന്ത്:**
- പ്രീ-ട്രെയിൻ ചെയ്ത മോഡൽ എങ്ങനെ ലോഡ് ചെയ്ത് ഉപയോഗിക്കാമെന്ന്
- ചിത്ര പ്രോസസ്സിംഗ്
- ചിത്രങ്ങളിൽ പ്രവചനങ്ങൾ നടത്തുന്നത്
- വിശ്വാസ സ്കോറുകൾ മനസിലാക്കൽ

**ഉപയോഗം:** ചിത്രങ്ങളിൽ ഉള്ള വസ്തുക്കൾ തിരിച്ചറിയുക (ഉദാഹരണത്തിന് "പൂച്ച", "നായ", "കാർ" തുടങ്ങിയവ) 

---


## ഘട്ടം 1: ആവശ്യമായ ലൈബ്രറികൾ ഇറക്കുമതി ചെയ്യുക

നമുക്ക് ആവശ്യമായ ഉപകരണങ്ങൾ ഇറക്കുമതി ചെയ്യാം. ഇതിൽ എല്ലാം ഇപ്പോൾ മനസ്സിലാകാതെ പേടിക്കേണ്ട!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## ഘട്ടം 2: മുൻകൂട്ടി പരിശീലിച്ച മോഡൽ ലോഡ് ചെയ്യുക

നാം **MobileNetV2** ഉപയോഗിക്കും, ഇത് മില്യണുകൾക്കണക്കിന് ചിത്രങ്ങളിൽ മുമ്പ് പരിശീലിപ്പിച്ച ഒരു ന്യൂറൽ നെറ്റ്‌വർക്ക് ആണ്.

ഇത് **ട്രാൻസ്ഫർ ലേണിംഗ്** എന്ന് വിളിക്കുന്നു - മറ്റൊരാൾ പരിശീലിപ്പിച്ച മോഡൽ ഉപയോഗിക്കുന്നത്!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## ഘട്ടം 3: സഹായക ഫംഗ്ഷനുകൾ

നമ്മുടെ മോഡലിനായി ചിത്രങ്ങൾ ലോഡ് ചെയ്ത് തയ്യാറാക്കാൻ ഫംഗ്ഷനുകൾ സൃഷ്ടിക്കാം.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## ഘട്ടം 4: സാമ്പിൾ ചിത്രങ്ങളിൽ പരീക്ഷിക്കുക

ഇന്റർനെറ്റിൽ നിന്നുള്ള ചില ചിത്രങ്ങൾ ക്ലാസിഫൈ ചെയ്യാൻ ശ്രമിക്കാം!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### ഓരോ ചിത്രവും വർഗ്ഗീകരിക്കുക


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## ഘട്ടം 5: നിങ്ങളുടെ സ്വന്തം ചിത്രങ്ങൾ പരീക്ഷിക്കുക!

താഴെ കൊടുത്തിരിക്കുന്ന URL നിങ്ങളുടെ ക്ലാസിഫൈ ചെയ്യാൻ ആഗ്രഹിക്കുന്ന ഏതെങ്കിലും ചിത്രത്തിന്റെ URL-ആയി മാറ്റുക.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 എന്താണ് ഇപ്പോൾ സംഭവിച്ചത്?

1. **നാം ഒരു മുൻകൂട്ടി പരിശീലിപ്പിച്ച മോഡൽ ലോഡ് ചെയ്തു** - MobileNetV2 മില്യണുകൾക്കണക്കിന് ചിത്രങ്ങളിൽ പരിശീലിപ്പിച്ചിരിക്കുന്നു  
2. **നാം ചിത്രങ്ങൾ മുൻപ്രോസസ്സ് ചെയ്തു** - മോഡലിനായി അവയുടെ വലിപ്പം മാറ്റി ഫോർമാറ്റ് ചെയ്തു  
3. **മോഡൽ പ്രവചനങ്ങൾ നടത്തി** - 1000 വസ്തു വർഗ്ഗങ്ങൾക്ക് സാധ്യതകൾ പുറത്തുവിട്ടു  
4. **നാം ഫലങ്ങൾ ഡികോഡ് ചെയ്തു** - സംഖ്യകൾ മനുഷ്യർക്ക് വായിക്കാൻ കഴിയുന്ന ലേബലുകളാക്കി മാറ്റി  

### ആത്മവിശ്വാസ സ്കോറുകൾ മനസ്സിലാക്കൽ

- **90-100%**: വളരെ ആത്മവിശ്വാസമുള്ളത് (ഏകദേശം തീർച്ചയായും ശരി)  
- **70-90%**: ആത്മവിശ്വാസമുള്ളത് (സാധാരണയായി ശരി)  
- **50-70%**: കുറച്ച് ആത്മവിശ്വാസമുള്ളത് (ശരി ആകാമെന്ന് തോന്നുന്നു)  
- **50% താഴെ**: അധികം ആത്മവിശ്വാസമില്ലാത്തത് (അസാധുത)  

### പ്രവചനങ്ങൾ തെറ്റായേക്കാനുള്ള കാരണങ്ങൾ?

- **അസാധാരണ കോണോ പ്രകാശനോ** - മോഡൽ സാധാരണ ഫോട്ടോകളിൽ പരിശീലിപ്പിച്ചിരിക്കുന്നു  
- **പല വസ്തുക്കൾ** - മോഡൽ ഒരു പ്രധാന വസ്തുവിനെ പ്രതീക്ഷിക്കുന്നു  
- **അസാധാരണ വസ്തുക്കൾ** - മോഡൽ 1000 വർഗ്ഗങ്ങൾ മാത്രമേ അറിയൂ  
- **താഴ്ന്ന ഗുണമേന്മയുള്ള ചിത്രം** - മങ്ങിയതോ പിക്‌സലേറ്റായതോ ആയ ചിത്രങ്ങൾ കഠിനമാണ്  

---


## 🚀 അടുത്ത ഘട്ടങ്ങൾ

1. **വിവിധ ചിത്രങ്ങൾ പരീക്ഷിക്കുക:**
   - [Unsplash](https://unsplash.com) ൽ ചിത്രങ്ങൾ കണ്ടെത്തുക
   - റൈറ്റ്-ക്ലിക്ക് → "Copy image address" തിരഞ്ഞെടുക്കുക URL ലഭിക്കാൻ

2. **പരീക്ഷണം നടത്തുക:**
   - ആബ്സ്ട്രാക്റ്റ് ആർട്ടുമായി എന്ത് സംഭവിക്കും?
   - വ്യത്യസ്ത കോണുകളിൽ നിന്നുള്ള വസ്തുക്കൾ തിരിച്ചറിയാൻ കഴിയും吗?
   - ഒരേ സമയം പല വസ്തുക്കളെ എങ്ങനെ കൈകാര്യം ചെയ്യുന്നു?

3. **കൂടുതൽ പഠിക്കുക:**
   - [Computer Vision lessons](../lessons/4-ComputerVision/README.md) പരിശോധിക്കുക
   - നിങ്ങളുടെ സ്വന്തം ഇമേജ് ക്ലാസിഫയർ പരിശീലിപ്പിക്കാൻ പഠിക്കുക
   - CNNs (Convolutional Neural Networks) എങ്ങനെ പ്രവർത്തിക്കുന്നു എന്ന് മനസിലാക്കുക

---

## 🎉 അഭിനന്ദനങ്ങൾ!

നിങ്ങൾ ഇപ്പോൾ ഒരു ആധുനിക ന്യൂറൽ നെറ്റ്‌വർക്കുപയോഗിച്ച് ഒരു ഇമേജ് ക്ലാസിഫയർ നിർമ്മിച്ചു!

ഈ തന്നെ സാങ്കേതിക വിദ്യ ഉപയോഗിക്കുന്നു:
- Google Photos (നിങ്ങളുടെ ഫോട്ടോകൾ ക്രമീകരിക്കുന്നത്)
- സ്വയം ഓടുന്ന കാറുകൾ (വസ്തുക്കൾ തിരിച്ചറിയൽ)
- മെഡിക്കൽ ഡയഗ്നോസിസ് (X-റേ വിശകലനം)
- ഗുണനിലവാര നിയന്ത്രണം (ദോഷങ്ങൾ കണ്ടെത്തൽ)

തുടർന്ന് അന്വേഷിക്കുകയും പഠിക്കുകയും ചെയ്യുക! 🚀


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ വ്യാഖ്യാനക്കേടുകൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
